# xgcm and croco run: 2 issues

1. When working on portions of datasets, xgrid need to be updated, otherwise it causes errors. I think this is so eversince we implemented the metrics.
2. One need to specify "boundary" to grid.interp, i.e. have kwargs in x2u, x2rho, ...

In [17]:
import crocosi.postp as pp
from xgcm import Grid

In [4]:
root_path = '/home/datawork-lops-osi/equinox/jetn/'
run = 'jet_cfg1_wp75_4km_1500a2000j_floats_lev50_itide/'

it = 1

r = pp.Run(root_path+run, prefix='file_',open_nc=['grid','his'], verbose=True)

# dataset
ds = r['his'].isel(time=it)

# segment of dataset
ds_trk =  r['his'].isel(time=it, y_rho=slice(100, 200))

Analysing directory /home/datawork-lops-osi/equinox/jetn/jet_cfg1_wp75_4km_1500a2000j_floats_lev50_itide/
Found 5 segments
Found 5 grid files
Found 25 his files
Detected time step of 300.0 s
Detected theta_s = 5.0
Detected theta_b = 0.0
Detected Hc = 100.0 m
Detected rho0 = 1000.0 kg/m^3
Detected H = 4000.0 m
Found 9 columns in output.mpi:
['STEP', 'time[DAYS]', 'KINETIC_ENRG', 'POTEN_ENRG', 'TOTAL_ENRG', 'NET_VOLUME', 'trd', 'ENSTROPHY', 'BTKIN_ENRG']
Opening NC datasets:  ['grid', 'his']
Grid size: (L ,M, N) = (258, 722, 50)


# 1) data set dimension size and xgrid
Interpolation works fine as long as you don't select a sub-portion of a dataset.

In [19]:
r.interp(ds.u_t_cos, "xi", boundary="extend") # need to specify boundaries, but that works

<xarray.DataArray 'mul-0b9d96694260e02bcf6ea7324b23ccf9' (s_rho: 50, y_rho: 722, x_rho: 258)>
dask.array<mul, shape=(50, 722, 258), dtype=float32, chunksize=(1, 722, 256), chunktype=numpy.ndarray>
Coordinates:
  * s_rho    (s_rho) float32 -0.99 -0.97 -0.95 -0.93 ... -0.07 -0.05 -0.03 -0.01
  * y_rho    (y_rho) float32 -2000.0 2000.0 6000.0 ... 2878000.0 2882000.0
  * x_rho    (x_rho) float32 -2000.0 2000.0 6000.0 ... 1022000.0 1026000.0

In [20]:
r.interp(ds_trk.u_t_cos, "xi", boundary="extend") # same

<xarray.DataArray 'mul-94f0c10d6e277cbbc9dd9becd5bb01d6' (s_rho: 50, y_rho: 100, x_rho: 258)>
dask.array<mul, shape=(50, 100, 258), dtype=float32, chunksize=(1, 100, 256), chunktype=numpy.ndarray>
Coordinates:
  * s_rho    (s_rho) float32 -0.99 -0.97 -0.95 -0.93 ... -0.07 -0.05 -0.03 -0.01
  * y_rho    (y_rho) float32 398000.0 402000.0 406000.0 ... 790000.0 794000.0
  * x_rho    (x_rho) float32 -2000.0 2000.0 6000.0 ... 1022000.0 1026000.0

In [9]:
r.interp(ds.v_t_cos, "eta", boundary="extend") # need to specify boundaries, but that works

<xarray.DataArray 'mul-dba5030d17bdcc0e47ece072a8ee0226' (s_rho: 50, y_rho: 722, x_rho: 258)>
dask.array<mul, shape=(50, 722, 258), dtype=float32, chunksize=(1, 720, 258), chunktype=numpy.ndarray>
Coordinates:
  * s_rho    (s_rho) float32 -0.99 -0.97 -0.95 -0.93 ... -0.07 -0.05 -0.03 -0.01
  * y_rho    (y_rho) float32 -2000.0 2000.0 6000.0 ... 2878000.0 2882000.0
  * x_rho    (x_rho) float32 -2000.0 2000.0 6000.0 ... 1022000.0 1026000.0

Here it crashes because of conflicting sizes. Note that if you interpolate v, which is on v-grid (and hence of size 721), it works

In [13]:
r.interp(ds_trk.T_t_cos, "eta", boundary="extend")

ValueError: conflicting sizes for dimension 'y_v': length 99 on the data but length 721 on coordinate 'y_v'

What if we make sure that ds_trk have corresponding sizes: same problem, and it still says that coordinates y_v has size 721. That is because it is the size in xgrid?

In [16]:
ds_trk = r['his'].isel(time=it, y_rho=slice(100, 200), y_v=slice(100, 199))
r.interp(ds_trk.T_t_cos, "eta", boundary="extend")

ValueError: conflicting sizes for dimension 'y_v': length 99 on the data but length 721 on coordinate 'y_v'

Fix xgrid -> then it works!

In [18]:
# copy coords and initialize new grid
gridcoord = {key:val.coords for key,val in r.xgrid.axes.items()}
grid = Grid(ds_trk, coords=gridcoord)
grid.interp(ds_trk.T_t_cos, "eta", boundary="extend")

<xarray.DataArray 'mul-28befca32125e7e25442f37de8c112a7' (s_rho: 50, y_v: 99, x_rho: 258)>
dask.array<mul, shape=(50, 99, 258), dtype=float32, chunksize=(1, 99, 258), chunktype=numpy.ndarray>
Coordinates:
  * s_rho    (s_rho) float32 -0.99 -0.97 -0.95 -0.93 ... -0.07 -0.05 -0.03 -0.01
  * y_v      (y_v) float32 400000.0 404000.0 408000.0 ... 788000.0 792000.0
  * x_rho    (x_rho) float32 -2000.0 2000.0 6000.0 ... 1022000.0 1026000.0

# 2) x2rho, ...

requires "boundary", so we should have kwargs. Maybe use defaults according to r.grid_periodicity ?

In [21]:
r.x2rho(ds.u_t_cos)

ValueError: `boundary` must be 'fill', 'extend' or 'extrapolate', not None.